Generation 과정 분산처리

In [1]:
import random
import torch
from Generation.generation_model import Generation_model
from utils import *
import asyncio
from transformers import AutoProcessor, AutoModelForCausalLM
from huggingface_hub import hf_hub_download
from PIL import Image
import torch
from transformers import AutoTokenizer, LlamaForCausalLM,LlamaTokenizer
from datasets import load_dataset

class Generation_model():
    def __init__(self, device) -> None:
        self.device = torch.device(f"cuda:{device}" if torch.cuda.is_available() else "cpu")
        self.model =  LlamaForCausalLM.from_pretrained("chavinlo/alpaca-native",low_cpu_mem_usage=True,torch_dtype='auto').to(self.device)
        self.tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")
        self.device_number = device
    def generate(self,input_samples):
        candidate_list = []
        #placeholders  = "'{}',"* 1
        #formatted_prompt = f"prompt promp" + placeholders + "PROMPT PROMPT {}".format(input_samples) 
        #print(input_samples)
        #for input_sample in input_samples:
            #prompt = formatted_prompt.format(*input_sample[1])
        formatted_prompt = input_samples
        inputs = self.tokenizer.encode(formatted_prompt,return_tensors="pt").to(self.device)
        candidate_senetnces = self.model.generate(inputs,max_new_tokens = 30) #make to candidate format
        text = self.tokenizer.decode(*candidate_senetnces)
       
        return text
        


/home/nlp-04/anaconda3/envs/kkm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
import random
import torch
from Generation.generation_model import Generation_model
from utils import *
import asyncio
from transformers import AutoProcessor, AutoModelForCausalLM
from huggingface_hub import hf_hub_download
from PIL import Image
import torch
from transformers import AutoTokenizer, LlamaForCausalLM,LlamaTokenizer
from datasets import load_dataset

model =  LlamaForCausalLM.from_pretrained("chavinlo/alpaca-native",low_cpu_mem_usage=True,torch_dtype='auto').to("cuda:0")
tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")

Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.34s/it]


pubmed20k

In [2]:

instruction = "Input sentences are part of a 'OBJECTIVE' in different medical papers. Please create five senetences in 'OBJECTIVE' part put new knowledge irrelevant with input sentences. Separate each sentence like 'sentence a:'"
input = """'To compare the effectiveness of adding cyclobenzaprine , tizanidine , or placebo to patient education and a self-care management program for patients with myofascial pain and specifically presenting with jaw pain upon awakening .','To evaluate the efficacy and long-term safety of linagliptin added to basal insulin in Asian patients with type 2 diabetes mellitus ( T2DM ) inadequately controlled by basal insulin with/without oral agents .'"""
prompt = f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
inputs = tokenizer.encode(prompt,return_tensors="pt").to("cuda")
candidate_senetnces = model.generate(inputs,max_new_tokens = 200,repetition_penalty = 1.3) #make to candidate format
text = tokenizer.decode(*candidate_senetnces)
print(text)

NameError: name 'tokenizer' is not defined

In [4]:
import re

def extract_sentences(text):
    sentences = re.split(r'sentence [A-Za-z0-9]+:', text, flags=re.IGNORECASE)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences
text = 'Sentence A1: To assess the effects of exercise on blood pressure control among hypertensive adults receiving antihypertensive medications. Sentence B1: The purpose of this study was to determine whether addition of metoprolol succinate to amlodipine besylate therapy would reduce ambulatory blood pressures more than monotherapy with either agent alone. Sentence C1: This trial evaluated the efficacy and safety of empagliflozin compared to glimepiride as add-on treatment to metformin in subjects with Type 2 Diabetes Mellitus. Sentence D1: We aimed to investigate the impact of lifestyle intervention plus pharmacological treatment versus pharmacological treatment only on glycaemic control in people with newly diagnosed type 2 diabetes. Sentence E1: Our objective was to examine the association between sleep duration and cardiov'
print(extract_sentences(text))

['To assess the effects of exercise on blood pressure control among hypertensive adults receiving antihypertensive medications.', 'The purpose of this study was to determine whether addition of metoprolol succinate to amlodipine besylate therapy would reduce ambulatory blood pressures more than monotherapy with either agent alone.', 'This trial evaluated the efficacy and safety of empagliflozin compared to glimepiride as add-on treatment to metformin in subjects with Type 2 Diabetes Mellitus.', 'We aimed to investigate the impact of lifestyle intervention plus pharmacological treatment versus pharmacological treatment only on glycaemic control in people with newly diagnosed type 2 diabetes.', 'Our objective was to examine the association between sleep duration and cardiov']


In [34]:
from transformers import pipeline

model_tag = "ml4pubmed/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext_pub_section"
classifier = pipeline(
              'text-classification', 
              model=model_tag, 
            )
            
prompt = """
This trial evaluated the efficacy and safety of empagliflozin compared to glimepiride as add-on treatment to metformin in subjects with Type 2 Diabetes Mellitus.
"""

classifier(
    prompt,
)

[{'label': 'METHODS', 'score': 0.48888838291168213}]

In [17]:

instruction = "this is sentence of medical domain paper. Change sentence of given sentence into 'mehtod' part sentence"
input = """'Inflammation plays a major role in the pathogenesis of rheumatoid arthritis; anti inflammatory drugs such as steroids have long been used to treat this condition.'"""
prompt = f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
inputs = tokenizer.encode(prompt,return_tensors="pt").to("cuda")
candidate_senetnces = model.generate(inputs,max_new_tokens = 200,repetition_penalty = 1.35) #make to candidate format
text = tokenizer.decode(*candidate_senetnces)
print(text)

  Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
this is sentence of medical domain paper. Change sentence of given sentence into'mehtod' part sentence

### Input:
'Inflammation plays a major role in the pathogenesis of rheumatoid arthritis; anti inflammatory drugs such as steroids have long been used to treat this condition.'

### Response:Methods 
• Inflammation plays a major role in the pathogenensis of rheumatoid arthritis.</s>


In [47]:

instruction = "input sentences are question about 'exchange rate', create five new question to ask about 'exchange rate',  Must be differenct with input setneces. Separate each sentence like 'sentence a:'"
input = """'Do you have the best exchange rate?','What is the basis of your exchange rates?'"""
prompt = f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
inputs = tokenizer.encode(prompt,return_tensors="pt").to("cuda")
candidate_senetnces = model.generate(inputs,max_new_tokens = 200,repetition_penalty = 1.2) #make to candidate format
text = tokenizer.decode(*candidate_senetnces)
print(text)

</s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
input sentences are question about 'exchange rate', create five new question to ask about 'exchange rate',  Must be differenct with input setneces. Separate each sentence like 'sentence a:'

### Input:
'Do you have the best exchange rate?','What is the basis of your exchange rates?'

### Response:Sentence A: Do you offer competitive exchange rates?
Sentence B: How do you determine your exchange rates?
Sentence C: What factors influence the fluctuation of exchange rates?
Sentence D: Are there any additional fees charged for currency conversions?
Sentence E: Is it possible to lock in an exchange rate?</s>


In [33]:
instruction = "The input sentence is one of the medical domain papers. what part of this sentence. choose one of this [OBJECTIVE, METHOD, BACKGROUND, RESULT, CONCLUSION]."
input = """'This trial evaluated the efficacy and safety of empagliflozin compared to glimepiride as add-on treatment to metformin in subjects with Type 2 Diabetes Mellitus.'"""
prompt = f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
inputs = tokenizer.encode(prompt,return_tensors="pt").to("cuda")
candidate_senetnces = model.generate(inputs,max_new_tokens = 10) #make to candidate format
text = tokenizer.decode(*candidate_senetnces)
print(text)

</s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
The input sentence is one of the medical domain papers. what part of this sentence. choose one of this [OBJECTIVE, METHOD, BACKGROUND, RESULT, CONCLUSION].

### Input:
'This trial evaluated the efficacy and safety of empagliflozin compared to glimepiride as add-on treatment to metformin in subjects with Type 2 Diabetes Mellitus.'

### Response:METHOD</s>


In [1]:
from Generation.generation_model import Generation_model
from utils import *
import asyncio
from transformers import AutoProcessor, AutoModelForCausalLM
from huggingface_hub import hf_hub_download
from PIL import Image
import torch
from transformers import AutoTokenizer, LlamaForCausalLM,LlamaTokenizer
from datasets import load_dataset

dataset = load_dataset("ml4pubmed/pubmed-classification-20k")
original_data = preprocess_dataset(dataset['train'])
few_dict = make_fewshot_dataset(original_data,2)
few_data = list(few_dict.items())
sublists_length = int(len(few_data)/2)
sublists = [few_data[i:i+sublists_length] for i in range(0, len(few_data), sublists_length)]


/home/nlp-04/anaconda3/envs/kkm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/home/nlp-04/.cache/huggingface/datasets/ml4pubmed___csv/ml4pubmed--pubmed-classification-20k-3de662485795ef5d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 549.21it/s]


In [9]:
print(few_dict)

{'OBJECTIVE': ['This study aimed to examine : ( 1 ) the quality of motor patterns of children with DCD participating in AVG by comparing them with children with typical development ( TD ) and ( 2 ) whether differences existed in the motor patterns utilized with 2 AVG types : Sony PlayStation 3 Move and Microsoft Xbox 360 Kinect .', 'We compared three methods of phototherapy for the treatment of moderate to severe facial acne vulgaris in Chinese patients .'], 'METHODS': ['Preparation of Anethum Graveolens boiled solution was as follows : 10g ( two tablespoons ) of seed in 100cc water boiled for 10min .', 'Pain ratings were obtained using the Faces Pain Scale-Revised , and conversation during the procedure was evaluated using the Child-Adult Medical Procedure Interaction Scale-Revised by 2 independent observers .'], 'RESULTS': ['Vorapaxar increased GUSTO moderate/severe bleeding numerically in medically managed patients ( adjusted HR 1.46 , 95 % CI 0.99-2 .15 ) .', 'At the end of experim

In [3]:
from dataclasses import dataclass, field
@dataclass
class generationArguments:
    model_name: str = field(
        default="chavinlo/alpaca-native",
        metadata = {
            "help": "The path for generation model",
        },
    )



In [5]:
b = generationArguments()
b.model_name

'chavinlo/alpaca-native'

In [2]:
async def generation_with_model_1(Generation_model = None):
    return Generation_model.generate(sublists[Generation_model.device_number])

models = [Generation_model(device_number) for device_number in range(2)]
async_list = [generation_with_model_1(model) for model in models]

Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.99s/it]


In [4]:
async def generation_with_model_1(Generation_model = None):
    return Generation_model.generate(sublists[Generation_model.device_number])
    
    await asyncio.sleep(0)
async_list = [generation_with_model_1(model) for model in models]
async def main():
    results = await asyncio.gather(
        *async_list
    )
    return results
    


result = await main()


[('OBJECTIVE', ['To assess the optimal scan delays and contrast injection durations for contrast-enhanced whole-body computed tomography ( CT ) .', 'The cross-sectional FOCUS ( Fixed-Dose Combination Drug for Secondary Cardiovascular Prevention ) study ( Phase 1 ) aimed to elucidate factors that interfere with appropriate adherence to CV medications for secondary prevention after an acute MI .']), ('METHODS', ['This large , multicenter , simulated-use study evaluated whether adults ( aged 18-65 years ) , caregivers ( parents/guardians aged 18-65 years of children aged 5-17 years ) , and children ( aged 11-17 years ) , with and without experience in using an EAI , had a preference for the current design of Auvi-Q or the current design of EpiPen .', 'Forty patients were treated with RF and 40 with Cryo .'])]


/home/nlp-04/anaconda3/envs/kkm/lib/python3.10/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[('RESULTS', ['We analysed the rates of stroke and systemic embolism in 6563 aspirin-treated patients with AF from the ACTIVE-A/AVERROES databases .', 'LPV/r-based ART was superior to NVP-based ART for efficacy and safety outcomes ; however , those on NVP had larger improvements in CD4 % , weight and height z-scores .']), ('CONCLUSIONS', ["OW individuals ' brain responses to food stimuli may indicate greater reward incentive processes than either SWL or NW groups .", 'Extended use of an alcohol-based mouthrinse is no more likely to cause reduction in salivary flow or perceived dryness in individuals with normal salivary flow compared with a non-alcohol-based mouthrinse ( CPH ) .'])]


In [5]:
print(result)

[[['OBJECTIVE', ' prompt promp\'{}\',PROMPT PROMPT [(\'OBJECTIVE\', [\'To assess the optimal scan delays and contrast injection durations for contrast-enhanced whole-body computed tomography ( CT ).\', \'The cross-sectional FOCUS ( Fixed-Dose Combination Drug for Secondary Cardiovascular Prevention ) study ( Phase 1 ) aimed to elucidate factors that interfere with appropriate adherence to CV medications for secondary prevention after an acute MI.\']), (\'METHODS\', [\'This large, multicenter, simulated-use study evaluated whether adults ( aged 18-65 years ), caregivers ( parents/guardians aged 18-65 years of children aged 5-17 years ), and children ( aged 11-17 years ), with and without experience in using an EAI, had a preference for the current design of Auvi-Q or the current design of EpiPen.\', \'Forty patients were treated with RF and 40 with Cryo.\'])]\n\ndef extract_paragraphs(paragraphs):\n    paragraphs_list = paragraphs.split("\\n\\n")\n    return'], ['METHODS', ' prompt prom

In [6]:
d = ['The median time to prostate-specific antigen progression was 4.7 months ( 95 % confidence interval 3.7-8 .3 ) ; the median overall survival was 11.8 months .', 'There was a weak correlation between prior contraceptive compliance and education level .']
print(d)
placeholders  = "'{}',"* 1
formatted_prompt = "prompt promp" + placeholders + "PROMPT PROMPT {}" 
prompt = formatted_prompt.format(*d)
print(prompt)

['The median time to prostate-specific antigen progression was 4.7 months ( 95 % confidence interval 3.7-8 .3 ) ; the median overall survival was 11.8 months .', 'There was a weak correlation between prior contraceptive compliance and education level .']
prompt promp'The median time to prostate-specific antigen progression was 4.7 months ( 95 % confidence interval 3.7-8 .3 ) ; the median overall survival was 11.8 months .',PROMPT PROMPT There was a weak correlation between prior contraceptive compliance and education level .


In [7]:

async def generation_with_model_1(Generation_model = None):
    print(Generation_model.generate("hello"))
async_list = [generation_with_model_1(model) for model in models]
b=  generation_with_model_1(model1)
c =  generation_with_model_1(model2)
async def main():
    await asyncio.gather(
        b,c
    )
    print("finsh")


await main()

NameError: name 'model1' is not defined

In [4]:

await main()

TypeError: An asyncio.Future, a coroutine or an awaitable is required

In [15]:
async_list = []
for a in range(2):
    globals()[f"hell{a}"] = generation_with_model_1(a)
    async_list.append(f"hell{a}")

/tmp/ipykernel_2558908/2895284244.py:3: RuntimeWarning: coroutine 'generation_with_model_1' was never awaited
  globals()[f"hell{a}"] = generation_with_model_1(a)


'hell'

In [1]:
from datasets import load_dataset

dataset = load_dataset("ml4pubmed/pubmed-classification-20k")

/home/nlp-04/anaconda3/envs/kkm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/home/nlp-04/.cache/huggingface/datasets/ml4pubmed___csv/ml4pubmed--pubmed-classification-20k-3de662485795ef5d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 469.55it/s]


In [52]:
labels = set()

# 유니크한 라벨 추출
for item in dataset['train']:
    label = item['label']
    labels.add(label)

labels = ['RESULTS','METHODS','OBJECTIVE','CONCLUSIONS']
# 라벨에 따라 요소를 분류할 딕셔너리 초기화
label_dict = {label: [] for label in labels}

# 리스트 순회하며 라벨에 따라 요소를 분류
for item in dataset['train']:
    label_dict[item['label']].append(item['text'])

# 결과 출력
for label, values in label_dict.items():
    print(label + ':', values)

KeyError: '###24854809'

In [45]:
label = list(set(get_labels(dataset)))

In [14]:
from datasets import load_dataset

dataset = load_dataset("ml4pubmed/pubmed-classification-20k")

Found cached dataset csv (/home/nlp-04/.cache/huggingface/datasets/ml4pubmed___csv/ml4pubmed--pubmed-classification-20k-3de662485795ef5d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 584.33it/s]


In [43]:
def preprocess_dataset(dataset):
# 레이블 정보 가져오기
    labels = dataset['label']  # 훈련 데이터셋의 레이블 가져오기

    # 레이블에 해당하는 인덱스 식별하기
    label_indices = {}
    for i, label in enumerate(labels):
        if label not in label_indices:
            label_indices[label] = []
        label_indices[label].append(i)

    # 레이블별로 데이터 저장할 리스트 생성
    label_lists = {}
    for label, indices in label_indices.items():
        if len(indices) > 1:  # 데이터가 있는 경우에만 리스트 생성
            label_lists[label] = []

    # 레이블별로 데이터 저장
    for label, indices in label_indices.items():
        if label in label_lists:  # 데이터가 있는 레이블에 대해서만 처리
            for index in indices:
                data = dataset[index]  # 데이터 가져오기 (여기서는 예시로 훈련 데이터셋 사용)
                label_lists[label].append(data['text'])
    return label_lists

In [48]:
b = preprocess_dataset(dataset['train'])

In [50]:
import random
def make_fewshot_dataset(label_lists,num_samples):
    fewshot_data = {}
    for label,all_data in label_lists.items():
        few_data = random.sample(all_data,num_samples)
        fewshot_data[label] = few_data
    return fewshot_data



In [66]:
few_data = list(make_fewshot_dataset(b,2).items())
sublists_length = 4 #class length / device lenght
sublists = [few_data[i:i+sublists_length] for i in range(0, len(few_data), sublists_length)]
sublists

[[('OBJECTIVE',
   ['To evaluate the efficacy of a combined neurolytic block of the celiac and superior hypogastric plexuses for incapacitating upper abdominal cancer pain .',
    'To investigate the role of preoperative biometry for selecting initial contact lens power .']),
  ('METHODS',
   ['Children aged 9-12years from both sexes were randomly assigned to one of six groups to receive plain milk , fortified milk , plain orange juice , fortified orange juice , supplement or placebo .',
    'Fasting blood samples were taken at baseline and after an 8-week intervention for quantification of related factors .']),
  ('RESULTS',
   ['No significant differences were found in the expression of sarcoplasmic reticulum Ca ( 2 + ) ATPase ( SERCA2a ) and ryanodine receptors ( RyRs ) .',
    'In an analysis controlling for level , degree , and number of anchors , tranexamic acid reduced drain output and total blood losses .']),
  ('CONCLUSIONS',
   ['However , there is a need for further research

In [7]:
from Selection import Train_Selection_model



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForCL: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForCL from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForCL from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForCL were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['mlp.dense.we

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 272.84it/s]


Dataset csv downloaded and prepared to /data/1_data_server/kkm/TAGSv2/data/csv/default-8d2b31c95c183785/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00,  5.27it/s]


In [9]:
Train_Selection_model.train_selection_model()

/home/nlp-04/anaconda3/envs/kkm/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
07/03/2023 11:55:20 - INFO - Selection.simcse.trainers -   ***** Running training *****
07/03/2023 11:55:20 - INFO - Selection.simcse.trainers -     Num examples = 275601
07/03/2023 11:55:20 - INFO - Selection.simcse.trainers -     Num Epochs = 3
07/03/2023 11:55:20 - INFO - Selection.simcse.trainers -     Instantaneous batch size per device = 8
07/03/2023 11:55:20 - INFO - Selection.simcse.trainers -     Total train batch size (w. parallel, distributed & accumulation) = 8
07/03/2023 11:55:20 - INFO - Selection.simcse.trainers -     Gradient Accumulation steps = 1
07/03/2023 11:55:20 - INFO - Selection.simcse.trainers -     Total optimization steps = 103353
You'

Step,Training Loss


KeyboardInterrupt: 

In [57]:
extension = 'csv'
datasets = load_dataset(extension, data_files='/data/1_data_server/kkm/TAGSv2/SimCSE/data/nli_for_simcse.csv', cache_dir="./data/")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


Found cached dataset csv (/data/1_data_server/kkm/TAGSv2/data/csv/default-8d2b31c95c183785/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [58]:
column_names = datasets["train"].column_names

In [59]:
column_names

['sent0', 'sent1', 'hard_neg']

In [4]:
from Selection import Train_Selection_model


In [5]:
selection = Train_Selection_model.selection_model()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForCL: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.pooler.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForCL from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForCL from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForCL were not initialized from the model checkpoint at bert-base-uncased and are

In [6]:
selection.train()

07/04/2023 14:40:10 - WARNING - datasets.builder -   Found cached dataset csv (/data/1_data_server/kkm/TAGSv2/data/csv/default-8d2b31c95c183785/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/275601 [00:00<?, ? examples/s]

/home/nlp-04/anaconda/envs/kkm/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
07/04/2023 14:40:38 - INFO - Selection.simcse.trainers -   ***** Running training *****
07/04/2023 14:40:38 - INFO - Selection.simcse.trainers -     Num examples = 275601
07/04/2023 14:40:38 - INFO - Selection.simcse.trainers -     Num Epochs = 3
07/04/2023 14:40:38 - INFO - Selection.simcse.trainers -     Instantaneous batch size per device = 8
07/04/2023 14:40:38 - INFO - Selection.simcse.trainers -     Total train batch size (w. parallel, distributed & accumulation) = 8
07/04/2023 14:40:38 - INFO - Selection.simcse.trainers -     Gradient Accumulation steps = 1
07/04/2023 14:40:38 - INFO - Selection.simcse.trainers -     Total optimization steps = 103353
You're

Step,Training Loss


KeyboardInterrupt: 

In [16]:
import csv

def create_contrastive_data(file_path, data_dict):
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile,delimiter='\t')
        writer.writerow(['Anchor', 'Positive', 'Negative'])

        for label, data_list in data_dict.items():
            for i, anchor in enumerate(data_list):
                positive_samples = data_list[:i] + data_list[i+1:]

                for negative_label, negative_data_list in data_dict.items():
                    if negative_label != label:
                        for negative in negative_data_list:
                            writer.writerow([anchor, positive_samples[0], negative])

create_contrastive_data('contrastive_data.tsv',few_dict )

In [3]:
from datasets import load_dataset
datasets =  load_dataset('csv',delimiter='\t',data_files='/data/1_data_server/kkm/TAGSv2/contrastive_data.tsv', cache_dir="./data/")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 644.09it/s]


Dataset csv downloaded and prepared to /data/1_data_server/kkm/TAGSv2/data/csv/default-b78a69759be7cf11/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 331.88it/s]


In [1]:
import random


class Generation_model():
    def __init__(self, device) -> None:
        self.device = torch.device(f"cuda:{device}" if torch.cuda.is_available() else "cpu")
        self.model =  LlamaForCausalLM.from_pretrained("chavinlo/alpaca-native",low_cpu_mem_usage=True,torch_dtype='auto').to(self.device)
        self.tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")
        self.device_number = device
    def generate(self,input_samples):
        candidate_list = []
        placeholders  = "'{}',"* 1
        formatted_prompt = f"prompt promp" + placeholders + "PROMPT PROMPT {}".format(input_samples) 
        print(input_samples)
        for input_sample in input_samples:
            prompt = formatted_prompt.format(*input_sample[1])
            inputs = self.tokenizer.encode(formatted_prompt,return_tensors="pt").to(self.device)
            candidate_senetnces = self.model.generate(inputs) #make to candidate format
            text = self.tokenizer.decode(*candidate_senetnces)
            candidate_list.append([input_sample[0],text])

        return candidate_list
        


In [1]:
from Generation.generation_model import Generation_model
from utils import *
import asyncio
from transformers import AutoProcessor, AutoModelForCausalLM
from huggingface_hub import hf_hub_download
from PIL import Image
import torch
from transformers import AutoTokenizer, LlamaForCausalLM,LlamaTokenizer
from datasets import load_dataset

dataset = load_dataset("ml4pubmed/pubmed-classification-20k")
original_data = preprocess_dataset(dataset['train'])
few_data = list(make_fewshot_dataset(original_data,2).items())
sublists_length = int(len(few_data)/2)
sublists = [few_data[i:i+sublists_length] for i in range(0, len(few_data), sublists_length)]

/home/nlp-04/anaconda3/envs/kkm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/home/nlp-04/.cache/huggingface/datasets/ml4pubmed___csv/ml4pubmed--pubmed-classification-20k-3de662485795ef5d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 446.46it/s]


In [2]:
async def generation_with_model_1(Generation_model = None):
    return Generation_model.generate(sublists[Generation_model.device_number])

models = [Generation_model(device_number) for device_number in range(2)]
async_list = [generation_with_model_1(model) for model in models]

Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.99s/it]


In [4]:
async def generation_with_model_1(Generation_model = None):
    return Generation_model.generate(sublists[Generation_model.device_number])
    
    await asyncio.sleep(0)
async_list = [generation_with_model_1(model) for model in models]
async def main():
    results = await asyncio.gather(
        *async_list
    )
    return results
    


result = await main()


[('OBJECTIVE', ['To assess the optimal scan delays and contrast injection durations for contrast-enhanced whole-body computed tomography ( CT ) .', 'The cross-sectional FOCUS ( Fixed-Dose Combination Drug for Secondary Cardiovascular Prevention ) study ( Phase 1 ) aimed to elucidate factors that interfere with appropriate adherence to CV medications for secondary prevention after an acute MI .']), ('METHODS', ['This large , multicenter , simulated-use study evaluated whether adults ( aged 18-65 years ) , caregivers ( parents/guardians aged 18-65 years of children aged 5-17 years ) , and children ( aged 11-17 years ) , with and without experience in using an EAI , had a preference for the current design of Auvi-Q or the current design of EpiPen .', 'Forty patients were treated with RF and 40 with Cryo .'])]


/home/nlp-04/anaconda3/envs/kkm/lib/python3.10/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[('RESULTS', ['We analysed the rates of stroke and systemic embolism in 6563 aspirin-treated patients with AF from the ACTIVE-A/AVERROES databases .', 'LPV/r-based ART was superior to NVP-based ART for efficacy and safety outcomes ; however , those on NVP had larger improvements in CD4 % , weight and height z-scores .']), ('CONCLUSIONS', ["OW individuals ' brain responses to food stimuli may indicate greater reward incentive processes than either SWL or NW groups .", 'Extended use of an alcohol-based mouthrinse is no more likely to cause reduction in salivary flow or perceived dryness in individuals with normal salivary flow compared with a non-alcohol-based mouthrinse ( CPH ) .'])]


In [5]:
print(result)

[[['OBJECTIVE', ' prompt promp\'{}\',PROMPT PROMPT [(\'OBJECTIVE\', [\'To assess the optimal scan delays and contrast injection durations for contrast-enhanced whole-body computed tomography ( CT ).\', \'The cross-sectional FOCUS ( Fixed-Dose Combination Drug for Secondary Cardiovascular Prevention ) study ( Phase 1 ) aimed to elucidate factors that interfere with appropriate adherence to CV medications for secondary prevention after an acute MI.\']), (\'METHODS\', [\'This large, multicenter, simulated-use study evaluated whether adults ( aged 18-65 years ), caregivers ( parents/guardians aged 18-65 years of children aged 5-17 years ), and children ( aged 11-17 years ), with and without experience in using an EAI, had a preference for the current design of Auvi-Q or the current design of EpiPen.\', \'Forty patients were treated with RF and 40 with Cryo.\'])]\n\ndef extract_paragraphs(paragraphs):\n    paragraphs_list = paragraphs.split("\\n\\n")\n    return'], ['METHODS', ' prompt prom

In [6]:
d = ['The median time to prostate-specific antigen progression was 4.7 months ( 95 % confidence interval 3.7-8 .3 ) ; the median overall survival was 11.8 months .', 'There was a weak correlation between prior contraceptive compliance and education level .']
print(d)
placeholders  = "'{}',"* 1
formatted_prompt = "prompt promp" + placeholders + "PROMPT PROMPT {}" 
prompt = formatted_prompt.format(*d)
print(prompt)

['The median time to prostate-specific antigen progression was 4.7 months ( 95 % confidence interval 3.7-8 .3 ) ; the median overall survival was 11.8 months .', 'There was a weak correlation between prior contraceptive compliance and education level .']
prompt promp'The median time to prostate-specific antigen progression was 4.7 months ( 95 % confidence interval 3.7-8 .3 ) ; the median overall survival was 11.8 months .',PROMPT PROMPT There was a weak correlation between prior contraceptive compliance and education level .


In [7]:

async def generation_with_model_1(Generation_model = None):
    print(Generation_model.generate("hello"))
async_list = [generation_with_model_1(model) for model in models]
b=  generation_with_model_1(model1)
c =  generation_with_model_1(model2)
async def main():
    await asyncio.gather(
        b,c
    )
    print("finsh")


await main()

NameError: name 'model1' is not defined

In [4]:

await main()

TypeError: An asyncio.Future, a coroutine or an awaitable is required

In [15]:
async_list = []
for a in range(2):
    globals()[f"hell{a}"] = generation_with_model_1(a)
    async_list.append(f"hell{a}")

/tmp/ipykernel_2558908/2895284244.py:3: RuntimeWarning: coroutine 'generation_with_model_1' was never awaited
  globals()[f"hell{a}"] = generation_with_model_1(a)


'hell'

In [1]:
from datasets import load_dataset

dataset = load_dataset("ml4pubmed/pubmed-classification-20k")

/home/nlp-04/anaconda3/envs/kkm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/home/nlp-04/.cache/huggingface/datasets/ml4pubmed___csv/ml4pubmed--pubmed-classification-20k-3de662485795ef5d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 469.55it/s]


In [52]:
labels = set()

# 유니크한 라벨 추출
for item in dataset['train']:
    label = item['label']
    labels.add(label)

labels = ['RESULTS','METHODS','OBJECTIVE','CONCLUSIONS']
# 라벨에 따라 요소를 분류할 딕셔너리 초기화
label_dict = {label: [] for label in labels}

# 리스트 순회하며 라벨에 따라 요소를 분류
for item in dataset['train']:
    label_dict[item['label']].append(item['text'])

# 결과 출력
for label, values in label_dict.items():
    print(label + ':', values)

KeyError: '###24854809'

In [45]:
label = list(set(get_labels(dataset)))

In [14]:
from datasets import load_dataset

dataset = load_dataset("ml4pubmed/pubmed-classification-20k")

Found cached dataset csv (/home/nlp-04/.cache/huggingface/datasets/ml4pubmed___csv/ml4pubmed--pubmed-classification-20k-3de662485795ef5d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 584.33it/s]


In [43]:
def preprocess_dataset(dataset):
# 레이블 정보 가져오기
    labels = dataset['label']  # 훈련 데이터셋의 레이블 가져오기

    # 레이블에 해당하는 인덱스 식별하기
    label_indices = {}
    for i, label in enumerate(labels):
        if label not in label_indices:
            label_indices[label] = []
        label_indices[label].append(i)

    # 레이블별로 데이터 저장할 리스트 생성
    label_lists = {}
    for label, indices in label_indices.items():
        if len(indices) > 1:  # 데이터가 있는 경우에만 리스트 생성
            label_lists[label] = []

    # 레이블별로 데이터 저장
    for label, indices in label_indices.items():
        if label in label_lists:  # 데이터가 있는 레이블에 대해서만 처리
            for index in indices:
                data = dataset[index]  # 데이터 가져오기 (여기서는 예시로 훈련 데이터셋 사용)
                label_lists[label].append(data['text'])
    return label_lists

In [48]:
b = preprocess_dataset(dataset['train'])

In [50]:
import random
def make_fewshot_dataset(label_lists,num_samples):
    fewshot_data = {}
    for label,all_data in label_lists.items():
        few_data = random.sample(all_data,num_samples)
        fewshot_data[label] = few_data
    return fewshot_data



In [66]:
few_data = list(make_fewshot_dataset(b,2).items())
sublists_length = 4 #class length / device lenght
sublists = [few_data[i:i+sublists_length] for i in range(0, len(few_data), sublists_length)]
sublists

[[('OBJECTIVE',
   ['To evaluate the efficacy of a combined neurolytic block of the celiac and superior hypogastric plexuses for incapacitating upper abdominal cancer pain .',
    'To investigate the role of preoperative biometry for selecting initial contact lens power .']),
  ('METHODS',
   ['Children aged 9-12years from both sexes were randomly assigned to one of six groups to receive plain milk , fortified milk , plain orange juice , fortified orange juice , supplement or placebo .',
    'Fasting blood samples were taken at baseline and after an 8-week intervention for quantification of related factors .']),
  ('RESULTS',
   ['No significant differences were found in the expression of sarcoplasmic reticulum Ca ( 2 + ) ATPase ( SERCA2a ) and ryanodine receptors ( RyRs ) .',
    'In an analysis controlling for level , degree , and number of anchors , tranexamic acid reduced drain output and total blood losses .']),
  ('CONCLUSIONS',
   ['However , there is a need for further research

In [43]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

Path = '/data/1_data_server/kkm/TAGSv2/result/checkpoint-50'
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
device = 'cuda'
model_for_compare = AutoModel.from_pretrained(Path).to(device=device, non_blocking=True)

texts = [
    "To evaluate the efficacy and long-term safety of linagliptin added to basal insulin in Asian patients with type 2 diabetes mellitus ( T2DM ) inadequately controlled by basal insulin with/without oral agents .",
    "To compare the effectiveness of adding cyclobenzaprine , tizanidine , or placebo to patient education and a self-care management program for patients with myofascial pain and specifically presenting with jaw pain upon awakening .",
    "This trial evaluated the efficacy and safety of empagliflozin compared to glimepiride as add-on treatment to metformin in subjects with Type 2 Diabetes Mellitus.",
    "We studied the analgesia provided by a lidocaine/tetracaine patch compared to injectable lidocaine during incision and drainage ( I&D ) of skin abscesses ."
]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device= device)


# Get the embeddings
with torch.no_grad():
    embeddings = model_for_compare(**inputs, output_hidden_states=True, return_dict=True).pooler_output.cpu()

# Calculate cosine similarities
# Cosine similarities are in [-1, 1]. Higher means more similar
cosine_sim_0_1 = 1 - cosine(embeddings[0], embeddings[1])
cosine_sim_0_2 = 1 - cosine(embeddings[0], embeddings[2])

print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[1], cosine_sim_0_1))
print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[2], cosine_sim_0_2))

Some weights of the model checkpoint at /data/1_data_server/kkm/TAGSv2/result/checkpoint-50 were not used when initializing BertModel: ['mlp.dense.weight', 'mlp.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /data/1_data_server/kkm/TAGSv2/result/checkpoint-50 and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Cosine similarity between "To evaluate the efficacy and long-term safety of linagliptin added to basal insulin in Asian patients with type 2 diabetes mellitus ( T2DM ) inadequately controlled by basal insulin with/without oral agents ." and "To compare the effectiveness of adding cyclobenzaprine , tizanidine , or placebo to patient education and a self-care management program for patients with myofascial pain and specifically presenting with jaw pain upon awakening ." is: 0.940
Cosine similarity between "To evaluate the efficacy and long-term safety of linagliptin added to basal insulin in Asian patients with type 2 diabetes mellitus ( T2DM ) inadequately controlled by basal insulin with/without oral agents ." and "This trial evaluated the efficacy and safety of empagliflozin compared to glimepiride as add-on treatment to metformin in subjects with Type 2 Diabetes Mellitus." is: 0.933


In [53]:
params = list(model_for_compare.parameters())[4]
params

Parameter containing:
tensor([-2.5578e-02, -1.9762e-02,  2.4464e-02,  8.9046e-02, -6.2867e-02,
        -1.3122e-02,  1.3072e-02, -1.2388e-02,  4.2745e-03,  3.5011e-02,
        -4.7748e-02, -1.3761e-02,  4.6714e-03, -1.0351e-01, -7.5751e-02,
         2.6834e-02, -2.0559e-02, -4.3415e-02, -5.6481e-02, -8.6584e-02,
        -8.3513e-03, -3.6880e-02, -4.3302e-02, -2.3370e-02,  1.9864e-02,
        -6.5447e-03, -6.1685e-02, -4.6689e-02,  3.5560e-03, -5.5016e-03,
        -4.0606e-02, -1.0464e-01, -3.0832e-02, -4.7563e-02, -5.1416e-02,
         5.2637e-02, -6.1255e-03, -6.3060e-02, -3.2766e-02, -1.0008e-02,
        -2.6092e-02,  3.3052e-02,  2.3757e-02, -3.7957e-02, -7.1284e-02,
         1.9527e-02, -4.7985e-02, -2.5503e-02, -1.0550e-01,  4.5951e-03,
        -2.8748e-02, -5.9268e-02,  1.2232e-02,  1.2567e-01, -5.6493e-02,
        -7.5983e-02, -3.8981e-02,  6.9706e-03, -6.4116e-02, -4.7138e-02,
        -2.1067e-02, -4.0381e-02, -7.6950e-02, -5.0533e-02,  3.0166e-02,
         2.6789e-02, -4.1366e

이밑으로 싹다 지워 도됨


In [ ]:
    datasets = load_dataset("ml4pubmed/pubmed-classification-20k")
    original_data = preprocess_dataset(datasets['train'])
    few_dict = make_fewshot_dataset(original_data,2)
    few_data = list(few_dict.items())
    sublists_length = int(len(few_data)/100)
    create_contrastive_data('contrastive_data.tsv', few_dict)

In [5]:
datasets = load_dataset("ml4pubmed/pubmed-classification-20k")

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2752.77it/s]


Dataset csv downloaded and prepared to /home/nlp-04/.cache/huggingface/datasets/ml4pubmed___csv/ml4pubmed--pubmed-classification-20k-3de662485795ef5d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 873.45it/s]


In [8]:
original_data = preprocess_dataset(datasets['train'])
original_data = make_fewshot_dataset(original_data,2)

In [9]:
original_data

{'OBJECTIVE': ['In the Age-Related Eye Disease Study 1 ( AREDS 1 ) , a reduced progression to late stage AMD was found using vitamin C , E , zinc , and - carotene .',
  'Hansen A/S ) ( hereafter , L. casei 431 ) on immune response to influenza vaccination and respiratory symptoms in healthy adults .'],
 'METHODS': ['The primary end point is the neurological outcome 6 months post-injury on the Glasgow Outcome Scale .',
  'The study is a randomized , controlled trial with a nonrandomized arm for patients who are eligible but decline randomization .'],
 'RESULTS': ['All incisions were Seidel negative .',
  'PDS patients required postoperative interventions 58.7 % of the time , including a 5-fluorouracil ( 5-FU ) injection ( 58.7 % ) , suture lysis ( 48.3 % ) , and needling ( 27.6 % ) .'],
 'CONCLUSIONS': ['The study confirms that actively contacting women and including a date for a screening visit , notably increased participation in the screening program .',
  'Standardized scanning and 

In [11]:
candidate_data = {key: None for key in original_data.keys()}

generated_data = {key: None for key in original_data.keys()}


In [12]:
candidate_data

{'OBJECTIVE': None,
 'METHODS': None,
 'RESULTS': None,
 'CONCLUSIONS': None,
 'BACKGROUND': None}

In [13]:
instruction = "Input sentences are part of a '{}' in different medical papers. Please create five senetences in '{}}' part put new knowledge irrelevant with input sentences. Separate each sentence like 'sentence a:'"
input = """'To compare the effectiveness of adding cyclobenzaprine , tizanidine , or placebo to patient education and a self-care management program for patients with myofascial pain and specifically presenting with jaw pain upon awakening .','To evaluate the efficacy and long-term safety of linagliptin added to basal insulin in Asian patients with type 2 diabetes mellitus ( T2DM ) inadequately controlled by basal insulin with/without oral agents .'"""
prompt = f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"